In [ ]:
import math
import os
import warnings
from typing import Dict, List, Tuple

import albumentations as A
import cv2
import numpy as np
import random
import torch
import tqdm
import wandb
from PIL import Image
from torch import nn

import matplotlib.pyplot as plt
import torchmetrics
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchvision import transforms
import albumentations.pytorch
from glob import glob
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2


warnings.filterwarnings("ignore")

В этот раз мы познакомимся с задачей детекции. Сделаем это, обучив модель YOLOv5. Это далеко не самая современная модель, но она достаточно близка к тем, которые используются сейчас.

Всего есть два типа моделей детекции: двухэтапные и одноэтапные. Несколько примеров моделей, использующих разные подходы:
- RCNN подход: Предполагает двухэтапный проход модели по изображению, где изначально выделяются зоны способами классического компьютерного зрения, а затем происходит уже знакомая нам классификация объектов в зонах и предсказание с помощью задачи регрессии координат box для объектов. 
- YOLO подход: Одноэтапный детектор удаляет процесс извлечения области интереса и напрямую классифицирует и регрессирует анкерные блоки-кандидаты (anchor-boxes). YOLO — это архитектура детекции, которая называется YOU ONLY LOOK ONCE.Она обучается от от начала до конца, для обработки изображения и прогнозирования ограничивающих рамок (BBox) и меток классов для каждой ограничивающей рамки напрямую. Наверное, наиболее популярная, также лицензия предполгает постоянное появление новых моделей
- Detr - также одноэтапный, однако использующий трансформерную архитектуру и особенный дизайн предсказывающей головы, чтобы уйти от необходимости предсказания анкерных блоков (что довольно затратно), предсказывая объекты сразу. Иронично, но в поздних версиях к анкерам вернулись.

![alt_text](../additional_materials/images/detection_stages.jfif)



In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

Для начала посмотрим, как модель строит свои предсказания

In [ ]:
import torch

model = torch.hub.load("ultralytics/yolov5", "yolov5s")
im = "https://ultralytics.com/images/zidane.jpg"

for f in "zidane.jpg", "bus.jpg":
    torch.hub.download_url_to_file("https://ultralytics.com/images/" + f, f)  # download 2 images
im1 = Image.open("zidane.jpg")  # PIL image
im2 = cv2.imread("bus.jpg")[..., ::-1]  # OpenCV image (BGR to RGB)

results = model([im1, im2], size=640)  # batch of images
results.print()
results.show()

results.xyxy[0]
results.pandas().xyxy[0]

Другой способ загрузить предобученную модель - загрузить с хаба YOLO. 

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

In [ ]:
results = model([im1, im2])  # batch of images

In [1]:
# Выведите новые результаты

In [2]:
import supervision as sv
detections = sv.Detections.from_ultralytics(result)
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)

annotated_image = image.copy()
annotated_image = box_annotator.annotate(annotated_image, detections=detections)
annotated_image = label_annotator.annotate(annotated_image, detections=detections)

sv.plot_image(annotated_image, size=(10, 10))

ModuleNotFoundError: No module named 'supervision'

Как устроена модель: 

- **Backbone** - это основная часть сети. Для YOLOv5 бекбон спроектирован с использованием CSP-Darknet53 — модификации архитектуры Darknet, использовавшейся в предыдущих версиях.
- **Neck**: Эта часть соединяет backbone и head. В YOLOv5 используются структуры SPPF и New CSP-PAN.
- **Head**: Эта часть отвечает за конечный результат. Она генерирует предсказанные bbox-ы.


У YOLO длинная история. Рассмотрим некоторые основные идеи. 
![alt_text](../additional_materials/images/yolo_evolution.png)

Мы сегодня рассмотрим основные вещи YOLOv5, попробуем ее и YOLOv11. 
Итак, общая архитектура YOLOv5 показана ниже:
![alt_text](../additional_materials/images/YOLOv5-1.png).

Какие важные идеи были использованы? 
Первое - SCPNet (Cross Stage Partial Network). Это важная часть, которая довольно долго тянется в YOLO для того, чтобы управлять протеканием градиентов. Чуть раньше перед этим была придумана DenseNet как ультра-версия резнета ( все блоки соединены со всеми), и из нее выросла идея конкатенировать выходы блоков с более поздними. Это было добавлено в YOLO4, показало эффективность и используется в некотором виде до сих пор. 

![alt_text](../additional_materials/images/yolo_back.png).

Второе - Spatial Pyramid pooling. Этот вид пулинга берет входы с разных этапов прохода по сети и объединяет ыместе конкатенацией, что похволяет находить объекты разных размеров.

![alt_text](../additional_materials/images/sppf.jfif).

Обучение проводится за один шаг, а не в несколько этапов. В YOLOv5 функция лосса была следующей:
![alt_text](../additional_materials/images/det_loss.svg)

Она содержит: 
- Classes Loss (BCE Loss): Бинарная кросс-энтропия для классификации категорий
- Objectness Loss (BCE Loss): Еще одна кросс-энтропия, которая позволяет определить, есть ли объект в предсказанном bbox-е.
- Location Loss (CIoU Loss): Complete IoU loss, определяет ошибку локализации в гриде
Кроме того, функция потерь взвешивается для разных размеров объектов: $L = w_{big} * L_{big} + w_{medium} * L_{medium} + w_{small} * L_{small} $



Как строятся обучающие датасеты? У YOLO свой формат,описывающий изображения следующим образом: 
Рассмотрим пример картинки: 

![alt_text](../additional_materials/images/two-persons-tie.avif)

Она содержит два объекта. Каждому из них соответствует bbox и класс. 
Они сохраняются в файл с разметкой вида: 
<img src="../additional_materials/images/two-persons-tie-1.avif" alt="drawing" width="500"/>

Датасет же организуется так как указано ниже:
```
../datasets/coco128/images/im0.jpg  # image
../datasets/coco128/labels/im0.txt  # label
```
Другой часто использующийся формат датасета - COCO. Пример ниже:
```json
{
    "info": {
        "description": "COCO 2017 Dataset",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2017,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
    },
    "licenses": [
        {"url": "http://creativecommons.org/licenses/by/2.0/","id": 4,"name": "Attribution License"}
    ],
    "images": [
        {"id": 242287, "license": 4, "coco_url": "http://images.cocodataset.org/val2017/xxxxxxxxxxxx.jpg", "flickr_url": "http://farm3.staticflickr.com/2626/xxxxxxxxxxxx.jpg", "width": 426, "height": 640, "file_name": "xxxxxxxxx.jpg", "date_captured": "2013-11-15 02:41:42"},
        {"id": 245915, "license": 4, "coco_url": "http://images.cocodataset.org/val2017/nnnnnnnnnnnn.jpg", "flickr_url": "http://farm1.staticflickr.com/88/xxxxxxxxxxxx.jpg", "width": 640, "height": 480, "file_name": "nnnnnnnnnn.jpg", "date_captured": "2013-11-18 02:53:27"}
    ],
    "annotations": [
        {"id": 125686, "category_id": 0, "iscrowd": 0, "segmentation": [[164.81, 417.51,......167.55, 410.64]], "image_id": 242287, "area": 42061.80340000001, "bbox": [19.23, 383.18, 314.5, 244.46]},
        {"id": 1409619, "category_id": 0, "iscrowd": 0, "segmentation": [[376.81, 238.8,........382.74, 241.17]], "image_id": 245915, "area": 3556.2197000000015, "bbox": [399, 251, 155, 101]},
        {"id": 1410165, "category_id": 1, "iscrowd": 0, "segmentation": [[486.34, 239.01,..........495.95, 244.39]], "image_id": 245915, "area": 1775.8932499999994, "bbox": [86, 65, 220, 334]}
    ],
    "categories": [
        {"supercategory": "speaker","id": 0,"name": "echo"},
        {"supercategory": "speaker","id": 1,"name": "echo dot"}
    ]
}
```

Как модели обучают? Можно, как обычно в torch, создать датасет и обучить модель, используя кастомные лоссы. Впрочем, в ultralitics есть готовые функции для этого, нужно лишь подготовить датасет.


In [ ]:
!pip install roboflow --user

Загрузим датасет. Это датасет, содержащий объекты и персонажей apex legends. Этот датасет - один из публичных датасетов roboflow, воспользуемся им.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR KEY")
project = rf.workspace("roboflow-100").project("apex-videogame")
version = project.version(2)
dataset = version.download("yolov5")

Дообучим модель. Обучать ее с нуля для нас нет смысла. Попробуем дообучить 2 эпохи.

In [ ]:
results = model.train(data="apex-videogame-2/data.yaml", epochs=2, imgsz=640)

Обратите внимание, как быстро обучается модель. Большой плюс YOLO - ее быстродействие. Результаты обучения сохраняются в файлы, их можно посмотреть после обучения. Посмотрим же. 

In [ ]:
from IPython.display import Image as IPyImage
filename = r"D:\projects\spbu_deep_learning\notebooks\runs\detect\train4\results.png"
IPyImage(filename=filename, width=600)

In [ ]:

filename = r"D:\projects\spbu_deep_learning\notebooks\runs\detect\train4\val_batch0_pred.jpg"
IPyImage(filename=filename, width=600)

Провалидируем модель: 

In [ ]:
!yolo task=detect mode=val model="D:\projects\spbu_deep_learning\notebooks\runs\detect\train4\weights\best.pt" data={dataset.location}/data.yaml

Задание: сделайте инференс модели на тестовых данных. Напишите функцию инференса и вывод результата